# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,959.105922,-0.204850,1234,958.901072,959.310772,1917.802143,1918.621544
6,279.159414,-0.501111,1234,278.658303,279.660524,557.316607,559.321049
7,751.504054,0.248843,1234,751.752897,751.255211,1503.505794,1502.510423
8,506.669770,0.487336,1234,507.157106,506.182433,1014.314212,1012.364867
9,550.484376,-0.699234,1234,549.785142,551.183610,1099.570285,1102.367220
10,535.210317,-0.897112,1234,534.313205,536.107429,1068.626410,1072.214858
11,197.800585,0.611700,1234,198.412285,197.188885,396.824570,394.377769
12,741.290321,-0.543694,1234,740.746627,741.834015,1481.493255,1483.668030
13,969.173154,0.989231,1234,970.162385,968.183923,1940.324770,1936.367847
14,725.754025,0.178577,1234,725.932601,725.575448,1451.865203,1451.150897


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-958.901072,959.310772
6,-278.658303,279.660524
7,-751.752897,751.255211
8,-507.157106,506.182433
9,-549.785142,551.183610
10,-534.313205,536.107429
11,-198.412285,197.188885
12,-740.746627,741.834015
13,-970.162385,968.183923
14,-725.932601,725.575448


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-958.901072,959.310772
6,-278.658303,279.660524
7,-751.752897,751.255211
8,-507.157106,506.182433
9,-549.785142,551.183610
10,-534.313205,536.107429
11,-198.412285,197.188885
12,-740.746627,741.834015
13,-970.162385,968.183923
14,-725.932601,725.575448


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-958.90107167,  959.31077223],
       [-278.65830328,  279.66052444],
       [-751.75289707,  751.25521147],
       [-507.15710617,  506.18243327],
       [-549.78514229,  551.18360987],
       [-534.31320479,  536.10742893],
       [-198.4122851 ,  197.18888472],
       [-740.74662736,  741.8340151 ],
       [-970.16238509,  968.18392332],
       [-725.93260144,  725.57544837]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1915.802143,-1918.621544
6,555.316607,-559.321049
7,1501.505794,-1502.510423
8,1012.314212,-1012.364867
9,1097.570285,-1102.367220
10,1066.626410,-1072.214858
11,394.824570,-394.377769
12,1479.493255,-1483.668030
13,1938.324770,-1936.367847
14,1449.865203,-1451.150897


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.959106,0.397575
6,0.279159,0.249445
7,0.751504,0.624421
8,0.506670,0.743668
9,0.550484,0.150383
10,0.535210,0.051444
11,0.197801,0.805850
12,0.741290,0.228153
13,0.969173,0.994615
14,0.725754,0.589288


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,959.105922,-0.204850
6,279.159414,-0.501111
7,751.504054,0.248843
8,506.669770,0.487336
9,550.484376,-0.699234
10,535.210317,-0.897112
11,197.800585,0.611700
12,741.290321,-0.543694
13,969.173154,0.989231
14,725.754025,0.178577


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
